In [2]:
import numpy as np

cl = np.load('cl_full.npy')[:,:,:]
model = np.load('model_full.npy')

kernel = ['none','10 arcmin','30 arcmin','1 deg']

def bin_corr(c_kq,l_min=100,l_max=1200,band=10):   # bin the power-spectrum

    width = int((l_max-l_min)/band)                    # number of l in each bin
    bin_Cl = np.zeros((band,1))
    bin_l = np.zeros((band,1))

    for i in range(band):
        ell_min = l_min+width*i
        ell_max = l_min+(i+1)*width
        ell_seq = np.arange(ell_min,ell_max,1)
        weights = (2*ell_seq+1)/np.sum(2*ell_seq+1)
        bin_l[i] = np.sum(weights*ell_seq)
        bin_Cl[i] = np.sum(weights*c_kq[ell_min:ell_max])

    return bin_l, bin_Cl

spectra = {j:[] for j in kernel}
for index,spec in enumerate(cl):
    for i in spec:
        bin_l,bin_cl = bin_corr(i,l_min=100,l_max=1200,band=10)
        spectra[kernel[index]].append(bin_cl)
    

bin_m = []
for ii in model:        
    bin_l, bin_model = bin_corr(ii,l_min=100,l_max=1200,band=10)
    bin_m.append(bin_model)

    
    
mean_spec = np.mean(np.array(bin_m),axis=0)
for i,sp in enumerate(spectra):
    mean = np.array(spectra[sp]).mean(0)
    var = np.mean(((mean_spec-mean)**2))
    print('%-9s %-3s : %e'%(kernel[i],'var',var))

none      var : 2.330797e-19
10 arcmin var : 2.076052e-19
30 arcmin var : 3.630893e-19
1 deg     var : 8.213063e-19
